In [22]:
import pandas as pd
import numpy as np
import gudhi as gd
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from itertools import combinations
import scipy.cluster.hierarchy as sch
from Bio import SeqIO
from Bio.Align import MultipleSeqAlignment
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Phylo import draw

In [23]:
import pandas as pd
import requests
from io import StringIO
from scipy.cluster.hierarchy import dendrogram, linkage
import matplotlib.pyplot as plt

# Define the URL
url = 'https://raw.githubusercontent.com/ccm-bioinfo/cambda2023/main/06_amr_resistance/data/230705_AMR_mysterious_NCBI_all_nelly.csv'

# Make a GET request to fetch the data from the URL
response = requests.get(url)

# Read the response content into a DataFrame
df = pd.read_csv(StringIO(response.text))
# Assuming you have a DataFrame named df and you want to remove a column named "included"
column_name = 'included'
# Remove the column from the DataFrame
df = df.drop(column_name, axis=1)
df.reset_index(drop=True, inplace=True)
Color = df['City'].map({'NYC': '#fb8072', 'BAL': '#8dd3c7','DEN':'#ffffb3','MIN': '#bebada','SAC':'#80b1d3','SAN':'#efa92a','mysterious':'#b3de69'})
# Inserting the new column as the second column
df.insert(1, 'Color', Color)

# Extract the columns for clustering and labeling
data = df.iloc[:, 6:].values
labels = df['Species'].values+"_"+df['City'].values+"_"+df['ID'].values
#---------------------------- 
Klebsiella = df[df['Species'] == 'Klebsiella pneumoniae'].reset_index(drop=True)
Enterobacter= df[df['Species'] == 'Enterobacter hormaechei'].reset_index(drop=True)
Escherichia = df[df['Species'] == 'Escherichia coli'].reset_index(drop=True)

#------------ sqaure matrix to TDA analysis
dist_matrix = pdist(df.iloc[:,6:].values, metric='hamming')
# Convert the pairwise distances to a square distance matrix
dist_matrix_square = squareform(dist_matrix)

distances=dist_matrix_square
threshold=.25
#print(distances.shape)
#print(data)
#print(df.iloc[0:145,:])
print("Klebsiella",Klebsiella.shape,"Enterobacter",Enterobacter.shape,"Escherichia",Escherichia.shape)

Klebsiella (225, 511) Enterobacter (28, 511) Escherichia (34, 511)


In [113]:
#Klebsiella['romb']

In [45]:
Klebsiella.iloc[212:226]
#213 Bal 215 NYC
#219 BAL 222 NYC
# Assuming you have a DataFrame called 'df'
row_index_1 = 213 # Bal
row_index_2 = 215 #NYC

# Extract the specific rows using iloc
row_1 = Klebsiella.iloc[row_index_1]
row_2 = Klebsiella.iloc[row_index_2]

# Assuming you have a DataFrame called 'df'
row_index_3 = 219
row_index_4 = 222

# Extract the specific rows using iloc
row_3 = Klebsiella.iloc[row_index_3] #CoBal
row_4 = Klebsiella.iloc[row_index_4] #co NYC



In [48]:
# Create boolean masks for the conditions
mask_row1_equals_1 = (row_1 == 1)  #BAL
mask_row2_equals_1 = (row_2 == 1)  #NYC
mask_row3_equals_1 = (row_3 == 1)  #CoBAL
mask_row4_equals_1 = (row_4 == 1)  #CoNYC


#print("NYC",row_1.iloc[20:60] )
print("BAl has",len(row_1.index[mask_row1_equals_1]),"non cero markers")
print("NYC has",len(row_2.index[mask_row2_equals_1]),"non cero markers")
print("Coassemblied BAl has",len(row_3.index[mask_row3_equals_1]),"non cero markers")
print("Coassemblied NYC has",len(row_4.index[mask_row4_equals_1]),"non cero markers")



BAl has 41 non cero markers
NYC has 145 non cero markers
Coassemblied BAl has 72 non cero markers
Coassemblied NYC has 201 non cero markers


In [56]:
columns_equal_to_1_in_row1_and_1_in_row2 = row_1.index[mask_row1_equals_1 & mask_row2_equals_1]
columns_equal_to_1_in_row3_and_1_in_row4 = row_1.index[mask_row3_equals_1 & mask_row4_equals_1]
# Print the columns that meet the conditions
print("Columns equal to 1 in row", row_index_1, "and equal to 1 in row", row_index_2, ":")
print("There are",len(columns_equal_to_1_in_row1_and_1_in_row2),"columns shared in" , row_index_1,"and", row_index_2)
#print(columns_equal_to_1_in_row1_and_1_in_row2)

# Print the columns that meet the conditions
print("Columns equal to 1 in row", row_index_3, "and equal to 1 in row", row_index_4, ":")
print("There are",len(columns_equal_to_1_in_row3_and_1_in_row4),"columns shared in" , row_index_3,"and", row_index_4)

Columns equal to 1 in row 213 and equal to 1 in row 215 :
There are 41 columns shared in 213 and 215
Columns equal to 1 in row 219 and equal to 1 in row 222 :
There are 71 columns shared in 219 and 222


In [60]:
mask_row2_equals_0 = (row_2 == 0)  #NYC
mask_row4_equals_0 = (row_4 == 0)  #coNYC

# Combine the masks to get the final result
columns_equal_to_1_in_row1_and_0_in_row2 = row_1.index[mask_row1_equals_1 & mask_row2_equals_0]
columns_equal_to_1_in_row3_and_0_in_row4 = row_3.index[mask_row3_equals_1 & mask_row4_equals_0]

# Print the columns that meet the conditions
print("Columns equal to 1 in row", row_index_1, "but equal to 0 in row", row_index_2, ":")
print(len(columns_equal_to_1_in_row1_and_0_in_row2))
# Print the columns that meet the conditions
print("Columns equal to 1 in row", row_index_3, "but equal to 0 in row", row_index_4, ":")
print(len(columns_equal_to_1_in_row3_and_0_in_row4),columns_equal_to_1_in_row3_and_0_in_row4)

Columns equal to 1 in row 213 but equal to 0 in row 215 :
0
Columns equal to 1 in row 219 but equal to 0 in row 222 :
1 Index(['roma'], dtype='object')


_romA_ is the only marker in BAL absent in NYC in _Klebsiella_

In [93]:
#Klebsiella['roma'==1]
# Filtering rows where the 'roma' column is equal to 1
sum(Klebsiella.iloc[1:124]['roma'] == 1) ## romA in Klebsiella mysterious sample
sum(Klebsiella['roma'] == 1) # romA in other samples

# Print the resulting DataFrame
#print(filtered_df)

65

Nevertheless all mysterious _Klebsiella_ sample does not contain _romA_, instead some genomes from NYC and Baltimore, and even NYC no coasemblied contains _romA_

# Enterobacter

In [99]:
row_index_1 = 16 # Bal
row_index_2 = 19 #NYC

# Extract the specific rows using iloc
row_1 = Enterobacter.iloc[row_index_1]
row_2 = Enterobacter.iloc[row_index_2]

# Assuming you have a DataFrame called 'df'
row_index_3 = 22
row_index_4 = 25

# Extract the specific rows using iloc
row_3 = Enterobacter.iloc[row_index_3] #CoBal
row_4 = Enterobacter.iloc[row_index_4] #co NYC

#Enterobacter
# Create boolean masks for the conditions
mask_row1_equals_1 = (row_1 == 1)  #BAL
mask_row2_equals_1 = (row_2 == 1)  #NYC
mask_row3_equals_1 = (row_3 == 1)  #CoBAL
mask_row4_equals_1 = (row_4 == 1)  #CoNYC


#print("NYC",row_1.iloc[20:60] )
print("BAl has",len(row_1.index[mask_row1_equals_1]),"non cero markers")
print("NYC has",len(row_2.index[mask_row2_equals_1]),"non cero markers")
print("Coassemblied BAl has",len(row_3.index[mask_row3_equals_1]),"non cero markers")
print("Coassemblied NYC has",len(row_4.index[mask_row4_equals_1]),"non cero markers")
mask_row2_equals_0 = (row_2 == 0)  #NYC
mask_row4_equals_0 = (row_4 == 0)  #coNYC

# Combine the masks to get the final result
columns_equal_to_1_in_row1_and_0_in_row2 = row_1.index[mask_row1_equals_1 & mask_row2_equals_0]
columns_equal_to_1_in_row3_and_0_in_row4 = row_3.index[mask_row3_equals_1 & mask_row4_equals_0]

# Print the columns that meet the conditions
print("Columns equal to 1 in row", row_index_1, "but equal to 0 in row", row_index_2, ":")
print(len(columns_equal_to_1_in_row1_and_0_in_row2))
# Print the columns that meet the conditions
print("Columns equal to 1 in row", row_index_3, "but equal to 0 in row", row_index_4, ":")
print(len(columns_equal_to_1_in_row3_and_0_in_row4),columns_equal_to_1_in_row3_and_0_in_row4)


BAl has 29 non cero markers
NYC has 119 non cero markers
Coassemblied BAl has 33 non cero markers
Coassemblied NYC has 171 non cero markers
Columns equal to 1 in row 16 but equal to 0 in row 19 :
0
Columns equal to 1 in row 22 but equal to 0 in row 25 :
0 Index([], dtype='object')


_Enterobacter_ has no markers in Baltimore that allow to differentiate them to NYC

In [107]:
Escherichia
row_index_1 = 22 # Bal
row_index_2 = 24 #NYC

# Extract the specific rows using iloc
row_1 = Escherichia.iloc[row_index_1]
row_2 = Escherichia.iloc[row_index_2]

# Assuming you have a DataFrame called 'df'
row_index_3 = 28
row_index_4 = 30

# Extract the specific rows using iloc
row_3 = Escherichia.iloc[row_index_3] #CoBal
row_4 = Escherichia.iloc[row_index_4] #co NYC

#Enterobacter
# Create boolean masks for the conditions
mask_row1_equals_1 = (row_1 == 1)  #BAL
mask_row2_equals_1 = (row_2 == 1)  #NYC
mask_row3_equals_1 = (row_3 == 1)  #CoBAL
mask_row4_equals_1 = (row_4 == 1)  #CoNYC


#print("NYC",row_1.iloc[20:60] )
print("BAl has",len(row_1.index[mask_row1_equals_1]),"non cero markers")
print("NYC has",len(row_2.index[mask_row2_equals_1]),"non cero markers")
print("Coassemblied BAl has",len(row_3.index[mask_row3_equals_1]),"non cero markers")
print("Coassemblied NYC has",len(row_4.index[mask_row4_equals_1]),"non cero markers")
mask_row2_equals_0 = (row_2 == 0)  #NYC
mask_row4_equals_0 = (row_4 == 0)  #coNYC

# Combine the masks to get the final result
columns_equal_to_1_in_row1_and_0_in_row2 = row_1.index[mask_row1_equals_1 & mask_row2_equals_0]
columns_equal_to_1_in_row3_and_0_in_row4 = row_3.index[mask_row3_equals_1 & mask_row4_equals_0]

# Print the columns that meet the conditions
print("Columns equal to 1 in row", row_index_1, "but equal to 0 in row", row_index_2, ":")
print(len(columns_equal_to_1_in_row1_and_0_in_row2),columns_equal_to_1_in_row1_and_0_in_row2)
# Print the columns that meet the conditions
print("Columns equal to 1 in row", row_index_3, "but equal to 0 in row", row_index_4, ":")
print(len(columns_equal_to_1_in_row3_and_0_in_row4),columns_equal_to_1_in_row3_and_0_in_row4)



BAl has 1 non cero markers
NYC has 91 non cero markers
Coassemblied BAl has 11 non cero markers
Coassemblied NYC has 180 non cero markers
Columns equal to 1 in row 22 but equal to 0 in row 24 :
1 Index(['entd'], dtype='object')
Columns equal to 1 in row 28 but equal to 0 in row 30 :
1 Index(['sul2_ARO:3000412'], dtype='object')


In [106]:
#Klebsiella['roma'==1]
# Filtering rows where the 'roma' column is equal to 1
sum(Escherichia['sul2_ARO:3000412'] == 1) ## romA in Klebsiella mysterious sample
#sum(Klebsiella['roma'] == 1) # romA in other samples

# Print the resulting DataFrame
print(Escherichia[Escherichia['sul2_ARO:3000412'] == 1]) ## romA in Klebsiella mysterious sample


                          ID    Color           Species        City  \
0                         S9  #b3de69  Escherichia coli  mysterious   
1                        S10  #b3de69  Escherichia coli  mysterious   
2                        S11  #b3de69  Escherichia coli  mysterious   
3                        S12  #b3de69  Escherichia coli  mysterious   
5                        S14  #b3de69  Escherichia coli  mysterious   
7                        S16  #b3de69  Escherichia coli  mysterious   
8                        S17  #b3de69  Escherichia coli  mysterious   
9                        S18  #b3de69  Escherichia coli  mysterious   
10                       S19  #b3de69  Escherichia coli  mysterious   
13                       S22  #b3de69  Escherichia coli  mysterious   
14  NYC_2016_GCF_002214205.1  #fb8072  Escherichia coli         NYC   
15  MIN_2017_GCF_013423565.1  #bebada  Escherichia coli         MIN   
16  MIN_2017_GCF_013423645.1  #bebada  Escherichia coli         MIN   
17  NY

In [108]:
#Klebsiella['roma'==1]
# Filtering rows where the 'roma' column is equal to 1
sum(Escherichia['entd'] == 1) ## romA in Klebsiella mysterious sample
#sum(Klebsiella['roma'] == 1) # romA in other samples

# Print the resulting DataFrame
print(Escherichia[Escherichia['entd'] == 1]) ## romA in Klebsiella mysterious sample


                          ID    Color           Species        City  \
1                        S10  #b3de69  Escherichia coli  mysterious   
2                        S11  #b3de69  Escherichia coli  mysterious   
4                        S13  #b3de69  Escherichia coli  mysterious   
5                        S14  #b3de69  Escherichia coli  mysterious   
6                        S15  #b3de69  Escherichia coli  mysterious   
7                        S16  #b3de69  Escherichia coli  mysterious   
9                        S18  #b3de69  Escherichia coli  mysterious   
10                       S19  #b3de69  Escherichia coli  mysterious   
12                       S21  #b3de69  Escherichia coli  mysterious   
13                       S22  #b3de69  Escherichia coli  mysterious   
14  NYC_2016_GCF_002214205.1  #fb8072  Escherichia coli         NYC   
15  MIN_2017_GCF_013423565.1  #bebada  Escherichia coli         MIN   
16  MIN_2017_GCF_013423645.1  #bebada  Escherichia coli         MIN   
17  NY